In [4]:
import numpy as np
from utils import load_episodes

episodes = load_episodes()

pairs = []

for episode in episodes:
    obs_seq = episode["observations"]
    health_seq = episode["health"]
    pairs.extend(list(zip(obs_seq, health_seq)))
    
pairs = np.array(pairs, dtype=object)
idx = np.random.permutation(len(pairs))
pairs = pairs[idx]

print(f"Total shuffled pairs: {len(pairs)}")

Total shuffled pairs: 9456


# Linear model

In [5]:
import numpy as np

X = []
y = []

for obs, health in pairs:
    X.append(obs.reshape(-1))      # (1,9) -> (9,)
    y.append(int(health < 3))      # 二分类标签

X = np.stack(X, axis=0)            # (N, 9)
y = np.array(y)                    # (N,)

print("Positive ratio:", y.mean()) # 看一下类别不平衡情况

Positive ratio: 0.24746192893401014


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

clf = LogisticRegression(
    max_iter=2000,
    n_jobs=-1
)

clf.fit(X_train, y_train)

c:\Users\ming\miniconda3\envs\FinRL\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [8]:
from joblib import dump
dump(clf, "data/linear.joblib")

['data/linear.joblib']

In [10]:
import joblib
clf = joblib.load("data/linear.joblib")

In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

acc = clf.score(X_test, y_test)
print(f"Test accuracy: {acc:.4f}")

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

y_prob = clf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print(f"ROC-AUC: {auc:.4f}")

coef = clf.coef_[0]
for i, w in enumerate(coef):
    print(f"dim {i}: {w:.3f}")

Test accuracy: 0.8932
              precision    recall  f1-score   support

           0     0.9279    0.9305    0.9292      1424
           1     0.7866    0.7799    0.7833       468

    accuracy                         0.8932      1892
   macro avg     0.8573    0.8552    0.8562      1892
weighted avg     0.8929    0.8932    0.8931      1892

ROC-AUC: 0.9481
dim 0: -1.492
dim 1: -1.521
dim 2: 0.357
dim 3: -0.695
dim 4: -0.709
dim 5: -1.378
dim 6: 0.604
dim 7: 0.000
dim 8: 0.645


In [17]:
prob = clf.predict_proba(X_test)[:, 1]
y_pred = (prob >= 0.5)
threshold = 0.3  # 试 0.3 / 0.25 / 0.2

y_pred = (prob >= threshold).astype(int)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9547    0.8722    0.9116      1424
           1     0.6920    0.8739    0.7724       468

    accuracy                         0.8726      1892
   macro avg     0.8233    0.8731    0.8420      1892
weighted avg     0.8897    0.8726    0.8771      1892



# MLP

In [18]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# X: (N, 9), y: (N,)  其中 y = (health < 3).astype(int)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y.numpy()
)

train_ds = TensorDataset(X_tr, y_tr)
test_ds  = TensorDataset(X_te, y_te)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False)

In [19]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)  # logits

In [20]:
# X: (N, 9), y: (N,)  其中 y = (health < 3).astype(int)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y.numpy()
)

train_ds = TensorDataset(X_tr, y_tr)
test_ds  = TensorDataset(X_te, y_te)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False)


device = "cuda" if torch.cuda.is_available() else "cpu"

model = MLP().to(device)

criterion = nn.BCEWithLogitsLoss()  # 二分类
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(150):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        logits = model(xb)
        loss = criterion(logits, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch:02d} | loss = {loss.item():.4f}")

C:\Users\ming\AppData\Local\Temp\ipykernel_29216\3848399420.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
C:\Users\ming\AppData\Local\Temp\ipykernel_29216\3848399420.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


Epoch 00 | loss = 0.5088
Epoch 20 | loss = 0.1016
Epoch 40 | loss = 0.2208
Epoch 60 | loss = 0.3419
Epoch 80 | loss = 0.2056
Epoch 100 | loss = 0.4085
Epoch 120 | loss = 0.2800
Epoch 140 | loss = 0.1116


In [21]:
from sklearn.metrics import roc_auc_score
model.eval()
with torch.no_grad():
    logits = model(X_te.to(device))
    probs = torch.sigmoid(logits).cpu().numpy()

auc = roc_auc_score(y_te.numpy(), probs)
print("MLP ROC-AUC:", auc)

# 还要测试recall, precision, f1-score等指标
from sklearn.metrics import classification_report

for thr in [0.5, 0.4, 0.3, 0.25, 0.2]:
    y_pred = (probs >= thr).astype(int)
    print(f"\nThreshold = {thr}")
    print(classification_report(
        y_te.numpy(), y_pred, digits=4
    ))

MLP ROC-AUC: 0.9486939402669741

Threshold = 0.5
              precision    recall  f1-score   support

         0.0     0.9273    0.9312    0.9292      1424
         1.0     0.7879    0.7778    0.7828       468

    accuracy                         0.8932      1892
   macro avg     0.8576    0.8545    0.8560      1892
weighted avg     0.8928    0.8932    0.8930      1892


Threshold = 0.4
              precision    recall  f1-score   support

         0.0     0.9417    0.9073    0.9242      1424
         1.0     0.7462    0.8291    0.7854       468

    accuracy                         0.8879      1892
   macro avg     0.8439    0.8682    0.8548      1892
weighted avg     0.8933    0.8879    0.8899      1892


Threshold = 0.3
              precision    recall  f1-score   support

         0.0     0.9562    0.8729    0.9126      1424
         1.0     0.6943    0.8782    0.7755       468

    accuracy                         0.8742      1892
   macro avg     0.8252    0.8755    0.8441  

# 

# The challenge

In [ ]:
import os
import sys
sys.path.append('./src')

import numpy as np
from dotenv import load_dotenv
from student_client import create_student_gym_env

def get_action(obs, repair_cnt):
    """
    obs: (9,) 原始观测
    返回: 0/1 代表健康/不健康
    """
    obs = normalize_obs(obs).reshape(1, -1)  # (1, 9)
    # use linear model
    prob = clf.predict_proba(obs)[0, 1]
    # prob是 0 1 2 轮死掉的概率，越大越不健康

    if prob >= 0.3:
        if repair_cnt < 4:
            return 1 # repair
        else:
            return 2 # 卖掉
    else:
        return 0 # do nothing

load_dotenv()

env = create_student_gym_env()
env.reset()

num_episodes = 20 

episode_total_rewards = []
episode_cum_rewards = []
episode_lengths = []
episode_repair_counts = []

for episode in range(num_episodes):
    print(f"\n--- Episode {episode + 1}/{num_episodes} ---")

    episode_observations = []
    episode_rewards = []
    episode_steps = []

    obs, info = env.reset()
    
    done = False
    
    repair_cnt = 0
    while not done:
        action = get_action(obs, repair_cnt)  # 根据当前观测和修复次数计算动作
        print(f"Action taken: {action} (0=healthy, 1=repair, 2=sell)", end=" | ")

        obs, reward, terminated, truncated, info = env.step(action)

        print(f"Reward={reward:.2f}, Total Episode Reward={sum(episode_rewards):.2f}")

        episode_rewards.append(reward)
        done = terminated or truncated

        if action == 1:  # If repair action was taken
            repair_cnt += 1

    total_reward = sum(episode_rewards)
    episode_total_rewards.append(total_reward)
    episode_cum_rewards.append(sum(episode_rewards))
    episode_lengths.append(len(episode_rewards))
    episode_repair_counts.append(repair_cnt)

2026-02-22 12:54:25,710 - student_client.student_gym_env - INFO - Client version 0.3 is newer than latest 0.2
2026-02-22 12:54:25,736 - student_client.student_gym_env - INFO - Created new session: 316154ab-8157-4ba4-b4cb-aa453a4e8ad8
2026-02-22 12:54:26,639 - student_client.student_gym_env - INFO - Created new episode: e423495a-a631-49f4-8fcc-848c70558a40
2026-02-22 12:54:26,640 - student_client.student_gym_env - INFO - StudentGymEnv initialized with episode e423495a-a631-49f4-8fcc-848c70558a40
2026-02-22 12:54:27,663 - student_client.student_gym_env - INFO - Episode e423495a-a631-49f4-8fcc-848c70558a40 reset successfully



--- Episode 1/20 ---


2026-02-22 12:54:28,583 - student_client.student_gym_env - INFO - Episode e423495a-a631-49f4-8fcc-848c70558a40 reset successfully


Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=427.61, Total Episode Reward=0.00
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=406.77, Total Episode Reward=427.61
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=379.51, Total Episode Reward=834.38
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=322.40, Total Episode Reward=1213.89
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=273.42, Total Episode Reward=1536.29
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=233.58, Total Episode Reward=1809.71
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=225.23, Total Episode Reward=2043.29
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=174.79, Total Episode Reward=2268.52
Action taken: 1 (0=healthy, 1=repair, 2=sell) | Reward=-541.70, Total Episode Reward=2443.31
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=419.18, Total Episode Reward=1901.61
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=355.20, Total Episode Reward=

2026-02-22 12:55:10,449 - student_client.student_gym_env - INFO - Episode e423495a-a631-49f4-8fcc-848c70558a40 reset successfully


...

Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=435.03, Total Episode Reward=0.00
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=392.70, Total Episode Reward=435.03
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=382.07, Total Episode Reward=827.73
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=341.42, Total Episode Reward=1209.80
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=296.26, Total Episode Reward=1551.22
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=258.89, Total Episode Reward=1847.49
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=187.71, Total Episode Reward=2106.38
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=148.58, Total Episode Reward=2294.09
Action taken: 1 (0=healthy, 1=repair, 2=sell) | Reward=-529.64, Total Episode Reward=2442.67
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=413.84, Total Episode Reward=1913.03
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=360.78, Total Episode Reward=

2026-02-22 13:06:57,221 - student_client.student_gym_env - INFO - Episode e423495a-a631-49f4-8fcc-848c70558a40 reset successfully


Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=451.77, Total Episode Reward=0.00
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=407.30, Total Episode Reward=451.77
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=384.66, Total Episode Reward=859.07
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=362.86, Total Episode Reward=1243.74
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=312.52, Total Episode Reward=1606.59
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=271.97, Total Episode Reward=1919.11
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=241.27, Total Episode Reward=2191.08
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=192.68, Total Episode Reward=2432.35
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=177.33, Total Episode Reward=2625.04
Action taken: 1 (0=healthy, 1=repair, 2=sell) | Reward=-510.56, Total Episode Reward=2802.37
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=400.26, Total Episode Reward=

2026-02-22 13:08:06,760 - student_client.student_gym_env - INFO - Episode e423495a-a631-49f4-8fcc-848c70558a40 reset successfully


Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=434.64, Total Episode Reward=0.00
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=428.78, Total Episode Reward=434.64
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=385.34, Total Episode Reward=863.42
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=346.04, Total Episode Reward=1248.76
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=320.10, Total Episode Reward=1594.80
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=268.33, Total Episode Reward=1914.90
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=223.00, Total Episode Reward=2183.23
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=190.07, Total Episode Reward=2406.23
Action taken: 1 (0=healthy, 1=repair, 2=sell) | Reward=-527.00, Total Episode Reward=2596.30
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=401.46, Total Episode Reward=2069.31
Action taken: 0 (0=healthy, 1=repair, 2=sell) | Reward=339.30, Total Episode Reward=